# Geolocalización IGR+Personas

En esta notebook haremos un intento de geolocalización con los textos de los usuarios... pero usando la medida IGR de Han, Cook y compañía. Pero usada con personas!


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("data/geoloc/users_train.json")
df_test = pd.read_json("data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [3]:
%%time
from contrastes.processing import build_dataframe_from_users

word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

CPU times: user 7min 18s, sys: 436 ms, total: 7min 18s
Wall time: 7min 18s


In [4]:
from contrastes.processing import preprocess_raw_df

word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


In [15]:
from contrastes.igr import igr

word_df["igr"] = igr(word_df, word_df.columnas_personas)

In [16]:
word_df.sort_values("igr", ascending=False, inplace=True)

word_df[:30][word_df.columnas_palabras]

,buenosaires_ocurrencias,catamarca_ocurrencias,chaco_ocurrencias,chubut_ocurrencias,cordoba_ocurrencias,corrientes_ocurrencias,entrerios_ocurrencias,formosa_ocurrencias,jujuy_ocurrencias,lapampa_ocurrencias,...,neuquen_ocurrencias,rionegro_ocurrencias,salta_ocurrencias,sanjuan_ocurrencias,sanluis_ocurrencias,santacruz_ocurrencias,santafe_ocurrencias,santiago_ocurrencias,tierradelfuego_ocurrencias,tucuman_ocurrencias
bombola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
logroño,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sanagasta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bombolo,0.0,0.0,4.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aijue,0.0,0.0,0.0,0.0,0.0,2.0,1.0,237.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mitai,0.0,0.0,0.0,0.0,0.0,7.0,0.0,123.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aij,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
unse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0
quintela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [12]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)
y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [13]:
%%time 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer


tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
liw_vectorizer = CountVectorizer(
    tokenizer=tokenizer.tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 9min 54s, sys: 392 ms, total: 9min 54s
Wall time: 9min 54s


Ya las tenemos vectorizadas en el orden esperado!

In [14]:
from sklearn.linear_model import LogisticRegression

clfs = {}
scores = {}

In [15]:


for num_words in range(250, 5000, 250):    
    if num_words in clfs:
        print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
        continue
    X_tr = X_train[:, :num_words].todense()
    X_tst = X_test[:, :num_words].todense()
    
    clf = LogisticRegression(
        multi_class='multinomial', solver='saga', penalty='l2', 
        max_iter=200, n_jobs=-1)
    clf.fit(X_tr, y_train)
    
    scores[num_words] = clf.score(X_tst, y_test)
    print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
    clfs[num_words] = clf
    

/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


250 palabras ----> accuracy 32.96


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


500 palabras ----> accuracy 40.52


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


...

/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


4250 palabras ----> accuracy 68.88


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


4500 palabras ----> accuracy 70.20
4750 palabras ----> accuracy 70.12


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


2500 palabras dan un accuracy de 71%. BASTANTE BIEN. Luego disminuye la performance

In [19]:
for num_words in range(5000, 20000, 500):
    if num_words in clfs:
        print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
        continue
    
    X_tr = X_train[:, :num_words].todense()
    X_tst = X_test[:, :num_words].todense()
    
    clf = LogisticRegression(
        multi_class='multinomial', solver='saga', penalty='l2', 
        max_iter=200, n_jobs=-1)
    clf.fit(X_tr, y_train)
    
    scores[num_words] = clf.score(X_tst, y_test)
    print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
    clfs[num_words] = clf

5000 palabras ----> accuracy 70.68
5500 palabras ----> accuracy 71.00
6000 palabras ----> accuracy 71.04
6500 palabras ----> accuracy 71.56
7000 palabras ----> accuracy 72.08
7500 palabras ----> accuracy 72.52
8000 palabras ----> accuracy 72.44
8500 palabras ----> accuracy 73.88
9000 palabras ----> accuracy 73.44
9500 palabras ----> accuracy 73.08
10000 palabras ----> accuracy 73.64
10500 palabras ----> accuracy 71.68
11000 palabras ----> accuracy 72.00
11500 palabras ----> accuracy 72.00
12000 palabras ----> accuracy 72.44
12500 palabras ----> accuracy 72.32
13000 palabras ----> accuracy 72.48
13500 palabras ----> accuracy 72.60
14000 palabras ----> accuracy 72.84
14500 palabras ----> accuracy 72.40
15000 palabras ----> accuracy 72.44
15500 palabras ----> accuracy 72.40
16000 palabras ----> accuracy 72.40
16500 palabras ----> accuracy 72.52
17000 palabras ----> accuracy 71.32
17500 palabras ----> accuracy 70.24
18000 palabras ----> accuracy 70.40


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


18500 palabras ----> accuracy 70.56


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


19000 palabras ----> accuracy 70.48


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


19500 palabras ----> accuracy 70.56


In [ ]:

scores

In [74]:
import pickle

pickle.dump(clfs, open("clfs_igr_personas.pkl", "wb"))
